In [2]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import powerlaw
from scipy.optimize import minimize
import plotly.graph_objects as go
import pandas as pd

In [6]:
lambda_param = 1.3
n = 1000 
data_exponencial = np.random.exponential(1/lambda_param, n)

In [7]:
mu = 0  # média
sigma = 1  # desvio padrão
data_normal = np.random.normal(mu, sigma, n)

In [8]:
mu_lognorm = 0  # média do logaritmo
sigma_lognorm = 1  # desvio padrão do logaritmo
data_lognormal = np.random.lognormal(mu_lognorm, sigma_lognorm, n)

In [13]:
def generate_power_law(alpha, xmin, size):
    r = np.random.uniform(0, 1, size)  # Gera números uniformemente distribuídos
    return xmin * (1 - r) ** (-1 / (alpha - 1))

alpha_powerlaw = 2.5
xmin = 1.0
data_powerlaw = generate_power_law(alpha_powerlaw, xmin, n)

# data_powerlaw = powerlaw.find_xmin()
# powerlaw.Distribution.generate_random(n=n)
# .PowerLaw(xmin=xmin, alpha=alpha_powerlaw).generate_random(n)

In [10]:
def generate_double_power_law(alpha1, alpha2, xmin, x_c, size):
    size1 = int(size / 2)  # Número de pontos antes de x_c
    size2 = size - size1   # Número de pontos depois de x_c
    
    # Gerar dados na primeira faixa (xmin a x_c)
    data1 = generate_power_law(alpha1, xmin, size1)
    
    # Gerar dados na segunda faixa (x_c em diante)
    data2 = generate_power_law(alpha2, x_c, size2)
    
    return np.concatenate([data1, data2])

# Parâmetros da Double Power-Law
alpha1 = 2.0
alpha2 = 3.0
xmin = 1.0
x_c = 10.0

# Gerar dados Double Power-Law
data_double_powerlaw = generate_double_power_law(alpha1, alpha2, xmin, x_c, n)

In [ ]:
def fit_exp(dados):
    return 1/np.mean(dados)

def fit_norm(dados):
    return np.mean(dados), np.std(dados)

def fit_lognorm(dados):
    log_dados = np.log(dados)
    return np.mean(log_dados), np.std(log_dados)

def fit_powerlaw(dados):
    powerlawfit = powerlaw.Fit(dados)
    return powerlawfit.alpha, powerlawfit.xmin

    # xmin = np.min(dados) ################################
    # alpha_est = 1 + len(dados) / np.sum(np.log(dados / xmin))
    # return alpha_est, xmin

    # R, p = results.distribution_compare('power_law', 'lognormal')

def fit_double_powerlaw(dados):
    return Exception('A Ser Implementado')